In [67]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [68]:
import cv2 
import pytesseract
import numpy as np
from pytesseract import Output
import imutils
import pandas as pd

## Processando a Imagem

In [69]:
# image = cv2.imread('cupom2.jpg')
# image = cv2.imread('NotaFiscal1.jpeg')
image = cv2.imread('NotaFiscal2.jpeg')

img = image.copy()
(H, W) = img.shape[:2]
#img = cv2.resize(image, (800, 600))
#cv2.imshow('img', edged)
#cv2.waitKey(0)

## Aplicando tons de cinza

In [70]:
rgb = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

## Aplicando Desfoque


In [71]:
#blurImg = cv2.GaussianBlur(rgb, (5, 5), 0)

## Dilate to combine adjacent text contours

In [72]:
#edged = cv2.Canny(blurImg, 60, 160)
threshold_img = cv2.threshold(rgb, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

#thresh = cv2.adaptiveThreshold(blurImg,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)
#kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
#dilate = cv2.dilate(thresh, kernel, iterations=4)

## Detectando Bordas

In [73]:
#cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#cnts = cnts[0] if len(cnts) == 2 else cnts[1]

## Função para encontrar contornos na imagem

In [74]:
# def encontrar_contornos(img):
#   conts = cv2.findContours(img, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
#   conts = imutils.grab_contours(conts)
#   conts = sorted(conts, key = cv2.contourArea, reverse = True)[:6]
#   return conts

# conts = encontrar_contornos(edged.copy())
# #conts

# for c in conts:
#   perimetro = cv2.arcLength(c, True)
#   aproximacao = cv2.approxPolyDP(c, 0.02 * perimetro, True)
#   if len(aproximacao) == 4:
#     maior = aproximacao
#     break
# maior

# cv2.drawContours(img, maior, -1, (120, 255, 0), 28)
# cv2.drawContours(img, [maior], -1, (120, 255, 0), 2)
# cv2.imshow('img', img)
# cv2.waitKey(0)

# def ordenar_pontos(pontos):
#   pontos = pontos.reshape((4,2))
#   pontos_novos = np.zeros((4, 1, 2), dtype=np.int32)

#   add = pontos.sum(1)
#   pontos_novos[0] = pontos[np.argmin(add)]
#   pontos_novos[2] = pontos[np.argmax(add)]

#   dif = np.diff(pontos, axis = 1)
#   pontos_novos[1] = pontos[np.argmin(dif)]
#   pontos_novos[3] = pontos[np.argmax(dif)]

#   return pontos_novos

# pontos_maior = ordenar_pontos(maior)
# print(pontos_maior)

# pts1 = np.float32(pontos_maior)
# pts1

# (H, W) = img.shape[:2]
# print(H, W)

# pts2 = np.float32([[0,0], [W, 0], [W, H], [0, H]])

# matriz = cv2.getPerspectiveTransform(pts1, pts2)
# matriz

## Transformação de perspectiva

In [75]:
# transform = cv2.warpPerspective(img, matriz, (W, H))

In [76]:
# cv2.imshow('img', transform)
# cv2.waitKey(0)

In [77]:
# conts = encontrar_contornos(edged.copy())

In [78]:
# cv2.drawContours(edged, maior, -1, (120, 255, 0), 28)
# cv2.drawContours(edged, [maior], -1, (120, 255, 0), 2)
# #cv2.imshow('img', img)
# #cv2.waitKey(0)

In [95]:
import re

#parametros para o tesseract
custom_config = 'psm 4'

details = pytesseract.image_to_data(threshold_img, output_type=Output.DICT, config=custom_config)
text = pytesseract.image_to_string(threshold_img, config=custom_config)
print(text)
alldata = pd.DataFrame(details)

#alldata.to_excel('MarksData.xlsx')


RTT CHC USSARCL Aa
200 KG RK 70,90
fd GUUOOODUSRZE PAD DF FERMENTALAN N
{ (GA 46,00
yuh GOOUREAEES GE NURAHOU LORTA GHEE 6
2,000) bits 10. 5u

— TUTAL RS
cartag de Credito Rs
Traca R¢

ib Spius R$ 2.73 Federal e R$ 7,80 Estadual
tunte: IBPT Soi Zell

CHBLOU VENDA 4153218
“#ixa: CALANOS — Cperader: CATNA?

SAT No. 000841543
O1/08/2021 - 17:45:55

( a G10 S900 O84) 5430 84394 192] WIS]

| My

 

"yy
(#2, iO

1757

(#2 BQ
21,00

65,01
65 ty
0,00

 



In [80]:
def removeSpecialCharacters(string):
    alphanumeric = []

    for character in string:
        if character.isalnum():
            alphanumeric.append(character) 
        
    return alphanumeric

## Loop para pegar  Nome Estabelecimento

In [81]:
name = []
for i, data in enumerate(alldata['block_num']):
    if(data==3):
        if(alldata['line_num'][i]==1):    
            if(alldata['word_num'][i]>0):
                name.append(alldata['text'][i])


## Loop para Pegar Produtos

In [82]:
products = []
for i, data in enumerate(alldata['block_num']):
    if(data==5):
        if(alldata['line_num'][i]==1 and alldata['par_num'][i]==1):    
            if(alldata['word_num'][i]>0):
                products.append(alldata['text'][i])

## Loop para Pegar Total da Nota

In [83]:
total = []
for i, data in enumerate(alldata['block_num']):
    if(data==5):
        if(alldata['line_num'][i]==2 and alldata['par_num'][i]==1):    
            if(alldata['word_num'][i]>0):
                total.append(alldata['text'][i])

In [84]:
name = removeSpecialCharacters(name)
# products = removeSpecialCharacters(products)
# total = ''.join(total)
print(products)

def listToString(s): 
    
    # initialize an empty string
    str1 = "" 
    
    # traverse in the string  
    for ele in s: 
        str1 += ele  
    
    # return string  
    return str1 

['ib', 'Spius', 'R$', '2.73', 'Federal', 'e', 'R$', '7,80', 'Estadual']


## Criando Classe de Retorno

In [85]:
class Cupom(object):
    name = ""
    products
    total = ""

def make_cupom(name, products, major):
    cupom = Cupom()
    cupom.name = ' '.join(name)
    cupom.products = products
    cupom.total = total
    # Note: I didn't need to create a variable in the class definition before doing this.
    return cupom

cupoms = make_cupom(name, products, total)

print('Estabelecimento: '+cupoms.name)
print('Produtos:')
print(' '.join(cupoms.products))
print(' '.join(cupoms.total))


Estabelecimento: RS
Produtos:
ib Spius R$ 2.73 Federal e R$ 7,80 Estadual
tunte: IBPT Soi Zell


In [86]:
# print(details.keys())
# print('conf-----------------------------------------')
# print(details['conf'])
# print('top-----------------------------------------')
# print(details['top'])
# print('line_num-----------------------------------------')
# print(details['line_num'])
# print('word_num-----------------------------------------')
# print(details['word_num'])
# print('left-----------------------------------------')
# print(details['left'])
# print('top-----------------------------------------')
# print(details['top'])
# print('width-----------------------------------------')
# print(details['width'])
# print('height-----------------------------------------')
# print(details['height'])


In [87]:
n_boxes = len(details['text'])


for i in range(n_boxes):
    if int(details['conf'][i]) > 30:
        (x, y, w, h) = (details['left'][i], details['top'][i], details['width'][i], details['height'][i])
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

# cv2.imshow('img', img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [88]:
parse_text = []

word_list = []

last_word = []

for word in details['text']:
    if word.strip()!=''
        word_list.append(word)
        last_word = word
    if(last_word!='' and word =='') or (word==details['text'][-1]):
        parse_text.append(word_list)
        
        word_list = []
    

SyntaxError: invalid syntax (<ipython-input-88-9cd295ee784f>, line 8)

In [ ]:
import csv

with open('result_text.txt',  'w', newline="") as file:
    csv.writer(file, delimiter=" ").writerows(parse_text)